# Classificazione di immagini con Convolutional Neural Network (CNN)

In questa lezione, impariamo a risolvere un problema di classificazione tramite un approcchio di **Deep Learning** basato su una *Convolutional Neural Network* (CNN).

### Librerie
Importiamo subito le librerie che utilizzeremo nella nostra soluzione.

In particolare, la libreria `tensorflow` (con `keras`) è quella che ci permette di addestrare, testare e in generale gestire l'utilizzo delle reti neurali.

**NB** Un MLP può essere implementato sia in Tensorflow e Keras, che in Scikit-Learn (scorsa lezione).

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

### Dati


1.   Carica il file `.zip` contenente il dataset *Euclid_dataset_DL.zip* dataset (versione per il Deep Learning!) → questo lo useremo per il **training** e **validation**
2.   Carica il file `.zip` contenente il dataset *Euclid_dataset.zip* dataset ("standard" version) → questo lo useremo per il **testing**
3.   Estrai i dati con i comandi seguenti:

In [5]:
!unzip -q Euclid_dataset_DL.zip -d /content

checkdir:  cannot create extraction directory: /content
           Permission denied


In [3]:
!unzip -q Euclid_dataset.zip -d /content

checkdir:  cannot create extraction directory: /content
           Permission denied


### Caricamento dei dati

La libreria `keras` ci mette a disposizione un comodo metodo per importare senza fatica i nostri dati.

In questo caso, mettiamo **80% dei dati in training** e il rimanente **20% nel set di validazione**.

**Tools**:
   * `image_dataset_from_directory()`: crea un dataset utilizzando le immagini presenti nella cartella indicata, assegnando come etichette il nome delle cartelle presenti.

**Training** dataset

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='Euclid_dataset_DL',
    labels='inferred',
    label_mode='categorical',
    class_names=['triangle', 'rectangle', 'square', 'rhombus'],
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=1821,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

**Validation** dataset

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='Euclid_dataset_DL',
    labels='inferred',
    label_mode='categorical',
    class_names=['triangle', 'rectangle', 'square', 'rhombus'],
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=1821,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

**Testing** set

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='Euclid_dataset',
    labels='inferred',
    label_mode='categorical',
    class_names=['triangle', 'rectangle', 'square', 'rhombus'],
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=1821,
    validation_split=0,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

### Architettura della Rete Neurale (CNN)
E' tempo di definire quale modello di *Convolutional Neural Network* (CNN) andremo ad utilizzare.

Abbiamo due possibili scelte:

1.   Definire la **nostra architettura**.
2.   Usare una delle architetture proposte in letteratura.

Nella nostra esercitazione, utilizziamo `MobileNet`. Utilizziamo questa rete  **pre-trained** tramite il dataset `Imagenet` dataset (i pesi della rete sono scaricati dal server remoto).





In [ ]:
# Our model
model = tf.keras.applications.MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
    )

**Problema**: come puoi vedere, il modello pre-trained ha 1000 classi in totale, i.e. il layer finale ha 1000 neuroni, mentre il nostro problema di classificazione ha solo 4 classi.

E' necessario quindi adattare l'architettura. Possiamo rimuovere l'ultimo layer e rimpiazzarlo con uno con solo 4 neuroni definito da noi.

**Tools**:

*   [**Sequential model**](https://keras.io/guides/sequential_model/) in TensorFlow
  *   Possiamo definire una rete come sequenza di layer (`output = Layer()(input)`)
  *   I layer del modello sono accessibili con l'attributo `layers`
  *   Ogni layer ha un attributo `input` e `output`. Questi attributi possono essere utilzzati per una varietà di operazioni, tra cui modificare velocemente l'architettura di un modello.

In [ ]:
# Create a layer where input is the output of the second last layer
output = Dense(4, activation='softmax', name='predictions')(model.layers[-2].output)

# Then create the corresponding model
model = Model(model.input, output)

In [ ]:
model.summary()

### Parametri di training
Qui sono definiti:

*   Numero di **epoche**
*   Il salvataggio del modello
*   **Optimizer**
*   **Funzione obiettivo**

Possiamo definire anche le **callbacks**: una callback è un metodo che esegue una o più operazioni a un dato punto del training (*e.g.* all'inizio o alla fine di un'epoca, prima o dopo di un batch singolo, etc).

E' necessario **compilare** il modello prima della fase di training.



In [ ]:
epochs = 5

callbacks = [
    # to save the model after every epoch
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
    # logging
    tf.keras.callbacks.TensorBoard(log_dir="logs", write_graph=True, write_images=False, update_freq="epoch",)
]

model.compile(
    optimizer=keras.optimizers.SGD(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

### Training
Ora possiamo lanciare il nostro training: per fortuna tutta la complessità è gestita dalle librerie!

Se il training è (troppo) lento, ricorda di abilitare il supporto GPU: clicca su **Runtime → Change Runtime type → GPU** (dal menù a tendina)

In [ ]:
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

### Test del modello

Anche in questo caso possiamo utilizzare un metodo delle librerie.

E' importare specificare quale modello (ovvero quali pesi) utilizzare, visto che abbiamo i pesi della rete per ogni epoca.

In [ ]:
model_trained = keras.models.load_model('/content/save_at_4.h5')

model_trained.evaluate(
    x=test_ds,
    y=None,
    batch_size=32,
    verbose=True,
    sample_weight=None,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    return_dict=False,
)

Infine, possiamo abilitare il modulo di **TensorBoard** per vedere i grafici delle nostre loss!

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs

### Architettura personalizzata
E' anche possibile, come accennato prima, definire il proprio modello di rete convolutiva.

Proviamo a implementare una nostra rete prendendo ispirazione da AlexNet (https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf), uno delle prime architetture proposte in letteratura.

**NB** Se definisco una mia architettura, è improbabile trovare lo stesso modello pre-addestrato.

**NBB** Dopo aver definito l'architettura, è necessario compilare nuovamente il modello e modificare la dimensione delle immagini di input.

In [ ]:
# Import necessary components to build LeNet
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Normalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

def our_model(img_shape=(64, 64, 3), n_classes=4, weights=None):

	# Initialize model
  model = Sequential()

  model.add(Normalization(axis=-1))

	# layer 1
  model.add(Conv2D(30, (5, 5), input_shape=img_shape))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

	# layer 2
  model.add(Conv2D(30, (5, 5)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

	# layer 3
  model.add(Conv2D(30, (4, 4)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

	# layer 4
  model.add(Conv2D(30, (3, 3)))
  model.add(Activation('relu'))

	# layer 5
  model.add(Conv2D(120, (3, 3)))
  model.add(Activation('relu'))

	# flatten (layer 6)
  model.add(Flatten())

  # layer 7
  model.add(Dense(120))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))

	# layer 8
  model.add(Dense(84))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))

	# layer 9
  model.add(Dense(n_classes))
  model.add(Activation('softmax'))

  return model

model = our_model()

### Conclusioni
In questa esercitazione abbiamo addestrato un modello di rete neurale!

Il modello ha imparato da **solo** come **estrarre informazione** (*feature*) dalle immagini e come **risolvere il problema di classificazione** (grazie alla parte finale, il *classifier*).

Questo codice è solo il **punto di partenza**, il mondo del deep learning è molto vasto, con numerosi sviluppi avvenuti anche di recente (a CVPR 2024, una delle maggiori conferenze del settore, sono stati sottomessi circa 11500 articoli!).

Nel nostro codice si potrebbero inserire numerose varianti:

*   Definire un modello diverso
*   Modificare l'optimizer
*   Modificare l'ammontare dei dati di training
*   ... (la fantasia è l'unico limite!)

